In [1]:
import functools

from agents.data_cleaning_agent import DataCleaningAgent
from agents.human_agent import HumanAgent
from agents.labeling_agent import LabelingAgent
from agents.tweet_collector_agent import TweetCollectorAgent
from agents.generator_tweet_agent import GeneratorTweetAgent
from agents.dataset_preparator_agent import DatasetPreparatorAgent
from agents.classifier_agent import ClassifierAgent
from agents.test_model_agent import TestModelAgent
from agents.model_agent import ModelAgent


from langgraph.graph import StateGraph, START
from agents.supervisor_agent import SupervisorAgent
from langchain_core.messages import HumanMessage
from typing_extensions import TypedDict

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\salma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
class AgentState(TypedDict):
    messages: list[HumanMessage]
    next: str
    current_index: int
    data: dict
    context: str

In [3]:
members = ["Human", "TweetCollector", "DataCleaner", "Labeler", "Generator", "DatasetPreparator", "Classifier", "Tester"]
# members = ["Human", "Generator", "Modeling"]

In [4]:
tweet_collector = TweetCollectorAgent()
data_cleaner = DataCleaningAgent()
labeler = LabelingAgent()
human_agent = HumanAgent()
# modeling = ModelAgent(lang='french')
generator = GeneratorTweetAgent()
dataset_preparator = DatasetPreparatorAgent()
classifier = ClassifierAgent()
tester = TestModelAgent
supervisor = SupervisorAgent(members)

In [5]:
human_node = functools.partial(SupervisorAgent.agent_node, agent=human_agent, name="Human")
tweet_collector_node = functools.partial(SupervisorAgent.agent_node, agent=tweet_collector, name="TweetCollector")
data_cleaner_node = functools.partial(SupervisorAgent.agent_node, agent=data_cleaner, name="DataCleaner")
labeler_node = functools.partial(SupervisorAgent.agent_node, agent=labeler, name="Labeler")
generator_node = functools.partial(SupervisorAgent.agent_node, agent=generator, name="Generator")
dataset_preparator_node = functools.partial(SupervisorAgent.agent_node, agent=dataset_preparator, name="DatasetPreparator")
classifier_node = functools.partial(SupervisorAgent.agent_node, agent=classifier, name="Classifier")
tester_node = functools.partial(SupervisorAgent.agent_node, agent=tester, name="Tester")
# modeling_node = functools.partial(SupervisorAgent.agent_node, agent=modeling, name="Modeling")

In [6]:
workflow = StateGraph(AgentState)
workflow.add_node("Human", human_node)
workflow.add_node("TweetCollector", tweet_collector_node)
workflow.add_node("DataCleaner", data_cleaner_node)
workflow.add_node("Labeler", labeler_node)
workflow.add_node("Generator", generator_node)
workflow.add_node("DatasetPreparator", dataset_preparator_node)
workflow.add_node("Classifier", classifier_node)
workflow.add_node("Tester", tester_node)
# workflow.add_node("Modeling", modeling_node)
workflow.add_node("supervisor", supervisor.supervisor_agent)

In [7]:
# Ajout des arêtes entre agents et le superviseur
for member in members:
    workflow.add_edge(member, "supervisor")

# Ajouter des arêtes conditionnelles à partir du superviseur
workflow.add_conditional_edges("supervisor", SupervisorAgent.supervisor_decision, supervisor.conditional_map())

# Ajouter le point d'entrée dans le workflow (HumanAgent)
workflow.add_edge(START, "Human")

# Compiler le workflow pour l'exécution
graph = workflow.compile()

In [8]:
# Définition de la structure de l'état initial du workflow
initial_context_message = HumanMessage(
    content="Provide context for the tweets you want to collect. "
            "For example: 'Collect tweets related to Kamala Harris.'"
)

# Définir l'état initial avec le message de contexte
initial_state = {
    "messages": [
        initial_context_message
    ],
    "data": {},
}

# Définir une limite de récursion pour éviter des boucles infinies dans le workflow
execution_config = {"recursion_limit": 150}

# Démarrer le flux d'exécution en utilisant la méthode `stream` du graphe
events = graph.stream(
    initial_state,
    execution_config,
)

# Parcourir les événements générés par le flux et les afficher
for event in events:
    print(event)
    print("----")


--------------------
{'messages': [HumanMessage(content='Context provided by human:', additional_kwargs={}, response_metadata={})], 'data': '', 'context': 'Gouvernement Michel Barnier '}
--------------------
{'Human': {'messages': [HumanMessage(content='Context provided by human:', additional_kwargs={}, response_metadata={}, name='Human')], 'data': '', 'context': 'Gouvernement Michel Barnier '}}
----
Le superviseur a choisi : TweetCollector
{'supervisor': {'messages': [HumanMessage(content='Context provided by human:', additional_kwargs={}, response_metadata={}, name='Human'), HumanMessage(content='Superviseur: Prochaine action - TweetCollector', additional_kwargs={}, response_metadata={})], 'next': 'TweetCollector', 'current_index': 1, 'data': '', 'context': 'Gouvernement Michel Barnier '}}
----
--------------------
{'messages': [HumanMessage(content='Context provided by human:', additional_kwargs={}, response_metadata={}, name='Human'), HumanMessage(content='Superviseur: Prochaine ac

c:\Users\salma\OneDrive\Documents\URCA - M2\INFO0909\SMA-Twiter\sentimentAnalysis\agents\labeling_agent.py:23: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self.llm(prompt)


Query: Gouvernement Michel Barnier 
Query: Gouvernement Michel Barnier 
Query: Gouvernement Michel Barnier 
Query: Gouvernement Michel Barnier 
Query: Gouvernement Michel Barnier 
Query: Gouvernement Michel Barnier 
Query: Gouvernement Michel Barnier 
Query: Gouvernement Michel Barnier 
Query: Gouvernement Michel Barnier 
Query: Gouvernement Michel Barnier 
Query: Gouvernement Michel Barnier 
Query: Gouvernement Michel Barnier 
Query: Gouvernement Michel Barnier 
Query: Gouvernement Michel Barnier 
--------------------
{'messages': [HumanMessage(content="Action effectuée par l'agent DataCleaningAgent", additional_kwargs={}, response_metadata={}, name='DataCleaner'), HumanMessage(content='Superviseur: Prochaine action - Labeler', additional_kwargs={}, response_metadata={}), HumanMessage(content="Action effectuée par l'agent LabelingAgent", additional_kwargs={}, response_metadata={})], 'data': 'data/tweets_labeled.csv', 'context': 'Gouvernement Michel Barnier '}
--------------------
{'La

c:\Users\salma\OneDrive\Documents\URCA - M2\INFO0909\SMA-Twiter\sentimentAnalysis\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

...

 Trainable params: 2,055,683 (7.84 MB)

 Non-trainable params: 256 (1.00 KB)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 336ms/step - accuracy: 0.4197 - loss: 3.5085 - precision: 0.4197 - recall: 0.4197 - val_accuracy: 0.5000 - val_loss: 1.0867 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.4723 - loss: 1.7646 - precision: 0.4591 - recall: 0.4364 - val_accuracy: 0.6667 - val_loss: 1.0908 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.5457 - loss: 1.3827 - precision: 0.5176 - recall: 0.4861 - val_accuracy: 0.3333 - val_loss: 1.0940 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5165 - loss: 1.0485 - precision: 0.5188 - recall: 0.4944 - val_accuracy: 0.6667 - val_loss: 1.0886 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.1429 - loss: 1.1171 - precision: 0.0000e+00 - recall: 0.0000e+00


Loss: 1.12, Accuracy: 0.14, Precision: 0.00, Recall: 0.00
Modèle sauvegardé dans 'tweet_classifier.h5'.
--------------------
{'messages': [HumanMessage(content="Action effectuée par l'agent DatasetPreparatorAgent", additional_kwargs={}, response_metadata={}, name='DatasetPreparator'), HumanMessage(content='Superviseur: Prochaine action - Classifier', additional_kwargs={}, response_metadata={}), HumanMessage(content="Action effectuée par l'agent ClassifierAgent. Le modèle ANN est prêt et sauvegardé.", additional_kwargs={}, response_metadata={})], 'data': <keras.src.callbacks.history.History object at 0x0000023C456251C0>, 'context': 'Gouvernement Michel Barnier '}
--------------------
{'Classifier': {'messages': [HumanMessage(content="Action effectuée par l'agent ClassifierAgent. Le modèle ANN est prêt et sauvegardé.", additional_kwargs={}, response_metadata={}, name='Classifier')], 'data': <keras.src.callbacks.history.History object at 0x0000023C456251C0>, 'context': 'Gouvernement Miche

TypeError: TestModelAgent.invoke() missing 1 required positional argument: 'state'

In [ ]:
import pandas as pd
df = pd.read_csv("tweets_labeled.csv")
df

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    print("Erreur sur l'affichage")